Referência: https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#:~:text=Topic%20Modeling%20is%20a%20technique,in%20the%20Python%27s%20Gensim%20package.

In [1]:
# Bibliotecas
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
df = pd.read_csv('UkraineCombinedTweetsDeduped_FEB27.csv')
df=df[df.language=="en"]
df.head()

,Unnamed: 0,userid,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetid,tweetcreatedts,retweetcount,text,hashtags,language,coordinates,favorite_count,extractedts
0,1141800,1466752038960656385,Curtin2Tiffany,I am just like everyone else. The universe ex...,"Colorado, USA",253,40,94,2021-12-03 12:52:22.000000,1497724980259262467,2022-02-27 00:07:16,5,#UkraineRussiaWar Captured Russian soldiers\n\...,"[{'text': 'UkraineRussiaWar', 'indices': [19, ...",en,NaN,0,2022-02-27 00:09:33.084131
1,1141801,1111276809302216710,5ela60,الأب ..الأخ ..الجار ..الحبيب .. السديك,NaN,167,9,656,2019-03-28 14:40:12.000000,1497724980271984641,2022-02-27 00:07:16,28,Like everybody else I am rooting for Ukraine t...,[],en,NaN,0,2022-02-27 00:09:33.068011
3,1141803,597779527,OurTurnToRescue,"Issues: Threats to Democracy, Racism, GOP corr...",NaN,4847,4080,33666,2012-06-02 21:53:59.000000,1497724980573966346,2022-02-27 00:07:16,1,We elected a game show host and got a clown. #...,"[{'text': 'Ukraine', 'indices': [63, 71]}]",en,NaN,0,2022-02-27 00:09:33.051823
4,1141804,1646145848,DrWAVeSportCd1,"Addicted to News, Music, Cooking, Gardens, Out...",USA,5002,3950,466629,2013-08-04 21:07:08.000000,1497724980653694976,2022-02-27 00:07:16,277,Ukrainian soldiers wearing sunflowers in their...,"[{'text': 'Ukraine', 'indices': [122, 130]}, {...",en,NaN,0,2022-02-27 00:09:33.043812
5,1141805,2191952909,WyvernReports,"In war, truth is the first casualty. Focus #Sy...",The Blue Marble,700,6424,20689,2013-11-22 21:51:19.000000,1497724980833968129,2022-02-27 00:07:16,565,russian troops destroyed a commercial oil base...,[],en,NaN,0,2022-02-27 00:09:33.035798


In [7]:
# Convert to list
data = df.text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['#UkraineRussiaWar Captured Russian soldiers "They said what their own '
 'battalion, where they are from and then told him it was just training and '
 'did not know they are in ukraine." #RussianArmy #Soldiers #Україна #Ukraine '
 '#UkraineRussiaWar https://t.co/KVIPqs7TzJ https://t.co/LdlzDnxCzG']


In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['captured', 'russian', 'soldiers', 'they', 'said', 'what', 'their', 'own', 'battalion', 'where', 'they', 'are', 'from', 'and', 'then', 'told', 'him', 'it', 'was', 'just', 'training', 'and', 'did', 'not', 'know', 'they', 'are', 'in', 'ukraine', 'russianarmy', 'soldiers', 'украіна', 'ukraine', 'https', 'co', 'kvipqs', 'tzj', 'https', 'co', 'ldlzdnxczg']]


In [9]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['captured', 'russian', 'soldiers', 'they', 'said', 'what', 'their', 'own', 'battalion', 'where', 'they', 'are', 'from', 'and', 'then', 'told', 'him', 'it', 'was', 'just', 'training', 'and', 'did', 'not', 'know', 'they', 'are', 'in', 'ukraine', 'russianarmy', 'soldiers', 'украіна', 'ukraine', 'https', 'co', 'kvipqs', 'tzj', 'https', 'co', 'ldlzdnxczg']


In [10]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_words_bigrams[:1])

[['captured', 'russian', 'soldiers', 'said', 'battalion', 'told', 'training', 'know', 'ukraine', 'russianarmy', 'soldiers', 'украіна', 'ukraine', 'https', 'co', 'kvipqs', 'tzj', 'https', 'co', 'ldlzdnxczg']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_words_bigrams)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1)]]


In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.00292638319776

Coherence Score:  0.3554454326368252


In [25]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.121*"war" + 0.077*"ukrainewar" + 0.054*"ukrainerussia" + 0.033*"nuclear" '
  '+ 0.029*"forces" + 0.015*"stopputin" + 0.014*"towards" + 0.013*"stoprussia" '
  '+ 0.012*"put" + 0.012*"stopwar"'),
 (1,
  '0.091*"one" + 0.074*"help" + 0.052*"share" + 0.047*"weapons" + '
  '0.029*"everyone" + 0.028*"signed" + 0.024*"made" + 0.022*"call" + '
  '0.022*"please" + 0.019*"freedom"'),
 (2,
  '0.016*"find" + 0.014*"hours" + 0.013*"always" + 0.013*"history" + '
  '0.011*"order" + 0.011*"foreign" + 0.011*"already" + 0.010*"courage" + '
  '0.010*"mistake" + 0.009*"far"'),
 (3,
  '0.083*"world" + 0.074*"country" + 0.056*"president" + 0.052*"zelensky" + '
  '0.026*"international" + 0.024*"would" + 0.024*"go" + 0.023*"man" + '
  '0.021*"europe" + 0.020*"leader"'),
 (4,
  '0.033*"back" + 0.025*"new" + 0.025*"air" + 0.024*"children" + 0.023*"shot" '
  '+ 0.020*"fighter_jets" + 0.019*"make" + 0.019*"planes" + 0.015*"defense" + '
  '0.014*"two"'),
 (5,
  '0.278*"anonymous" + 0.093*"state" + 0.052*

In [24]:
#! pip install pyLDAvis

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models 
pyLDAvis.enable_notebook()
import matplotlib.pyplot as plt
%matplotlib inline

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.459504 -0.033902       1        1  25.416374
16    -0.045376  0.107119       2        1   6.925656
0     -0.005721  0.063144       3        1   6.028977
14     0.004752 -0.130787       4        1   5.716558
3     -0.007912  0.070132       5        1   5.161878
19     0.026197  0.075820       6        1   4.486988
11     0.038315  0.056129       7        1   4.230355
10     0.072747 -0.218461       8        1   4.151607
2      0.047353  0.082500       9        1   4.011242
18     0.035888  0.087051      10        1   3.998509
9      0.012248  0.070299      11        1   3.960995
15     0.041520  0.042479      12        1   3.665277
13     0.043495  0.053993      13        1   3.606586
8      0.012496  0.035832      14        1   3.510912
4      0.046798  0.061015      15        1   3.244766
1      0.067810 -0.059330      16        1   2.856011
7      0.044784  0.067499      17        1   2.796919
6      0.066086 -0.284464      18        1   2.464683
12     0.018879 -0.022842      19        1   1.911674
5     -0.060856 -0.123227      20        1   1.854032, topic_info=              Term           Freq          Total Category  logprob  loglift
14         ukraine  205400.000000  205400.000000  Default  30.0000  30.0000
2               co  177823.000000  177823.000000  Default  29.0000  29.0000
3            https  177691.000000  177691.000000  Default  28.0000  28.0000
28          russia   80752.000000   80752.000000  Default  27.0000  27.0000
49       ukrainian   34082.000000   34082.000000  Default  26.0000  26.0000
...            ...            ...            ...      ...      ...      ...
108074       nn_dw     472.371279     489.448607  Topic20  -5.0048   3.9523
108070  axcqwplxjm     472.371246     489.448574  Topic20  -5.0048   3.9523
108071        epyp     472.371246     489.448574  Topic20  -5.0048   3.9523
108072  gewxmfknfg     472.371246     489.448574  Topic20  -5.0048   3.9523
108062  uezkz_hhpb     543.620850     575.514718  Topic20  -4.8643   3.9308

[672 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
6491      17  0.998354         abandoned
2834      19  0.998683          absolute
60475     15  0.002827  absolute_fucking
60475     19  0.996203  absolute_fucking
481       18  0.999290            access
...      ...       ...               ...
988        8  0.998884             youre
306        5  0.999952          zelensky
716       12  0.999525         zelenskyy
897       18  0.999623            россия
15        16  0.999764           украіна

[906 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 17, 1, 15, 4, 20, 12, 11, 3, 19, 10, 16, 14, 9, 5, 2, 8, 7, 13, 6])